In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import binary_crossentropy
import numpy as np

# Define VAE Model
class VAE(Model):
    def __init__(self, latent_dim=20):
        super(VAE, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(400, activation='relu'),
            layers.Dense(latent_dim * 2)  # mean and log variance
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(400, activation='relu'),
            layers.Dense(784, activation='sigmoid')
        ])

    def reparameterize(self, z_mean, z_logvar):
        eps = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_logvar) * eps

    def call(self, x):
        x = self.encoder(x)
        z_mean, z_logvar = tf.split(x, num_or_size_splits=2, axis=1)
        z = self.reparameterize(z_mean, z_logvar)
        x_recon = self.decoder(z)
        return x_recon, z_mean, z_logvar

# Loss Function
def vae_loss(x, x_recon, z_mean, z_logvar):
    recon_loss = binary_crossentropy(x, x_recon) * 784
    kl_loss = -0.5 * tf.reduce_sum(1 + z_logvar - tf.square(z_mean) - tf.exp(z_logvar), axis=1)
    return tf.reduce_mean(recon_loss + kl_loss)

# Load Data
(train_images, _), _ = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 784).astype('float32') / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).batch(128)

# Train Model
vae = VAE()
optimizer = tf.keras.optimizers.Adam(1e-3)
for epoch in range(10):
    for x in train_dataset:
        with tf.GradientTape() as tape:
            x_recon, z_mean, z_logvar = vae(x)
            loss = vae_loss(x, x_recon, z_mean, z_logvar)
        grads = tape.gradient(loss, vae.trainable_variables)
        optimizer.apply_gradients(zip(grads, vae.trainable_variables))
    print(f"Epoch {epoch+1}, Loss: {loss.numpy():.4f}")

print("Training Complete!")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1, Loss: 137.9386
Epoch 2, Loss: 125.4968
Epoch 3, Loss: 119.6653
Epoch 4, Loss: 116.9123
Epoch 5, Loss: 115.1583
Epoch 6, Loss: 112.3009
Epoch 7, Loss: 112.0348
Epoch 8, Loss: 110.9090
Epoch 9, Loss: 109.3577
Epoch 10, Loss: 109.3811
Training Complete!
